### Querying Movie-Rating_Reviewer Database

You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema:

Movie ( mID, title, year, director )
English: There is a movie with ID number mID, a title, a release year, and a director.

Reviewer ( rID, name )
English: The reviewer with ID number rID has a certain name.

Rating ( rID, mID, stars, ratingDate )
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate.

Your queries will run over a small data set conforming to the schema. View the database. (You can also download the schema and data.)

Instructions: Each problem asks you to write a query in SQL.

In [ ]:
# import libraries
import pandas as pd
import sqlite3

In [ ]:
# connect to database
conn = sqlite3.connect(':memory:')

# create cursor object
cur = conn.cursor()

In [ ]:
# read in csv files to dataframe and write records 
# stored in the DataFrame to a SQL database

df1 = pd.read_csv('movie.csv', )
df1.to_sql('movie', con = conn, index=False)
df2 = pd.read_csv('rating.csv', )
df2.to_sql('rating', con = conn, index=False)
df3 = pd.read_csv('reviewer.csv', )
df3.to_sql('reviewer', con = conn, index=False)

In [ ]:
# view movie dataframe

cur.execute('''SELECT * FROM movie;''')            # sql query
movie = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
movie.columns = [x[0] for x in cur.description]    # labels dataframe columns
movie                                              # view dataframe

In [ ]:
# view rating dataframe

cur.execute('''SELECT * FROM rating;''')            # sql query
rating = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
rating.columns = [x[0] for x in cur.description]    # labels dataframe columns
rating                                              # view dataframe

In [ ]:
# view reviewer dataframe

cur.execute('''SELECT * FROM reviewer;''')            # sql query
reviewer = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
reviewer.columns = [x[0] for x in cur.description]    # labels dataframe columns
reviewer                                              # view dataframe